In [1]:
from presidio_analyzer import AnalyzerEngine, PatternRecognizer
from presidio_anonymizer import AnonymizerEngine
from presidio_analyzer.nlp_engine import NlpEngineProvider
from presidio_anonymizer.entities import RecognizerResult
from flair.data import Sentence
from flair.models import SequenceTagger
from transformers import pipeline
import pdfplumber
import stanza
import spacy

def extract_text_from_pdf(pdf_path):
    text = ""
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            page_text = page.extract_text()
            if page_text:
                text += page_text + "\n"
    return text.strip()

text = extract_text_from_pdf("NDA_Documents/NDA_Agreement_Ali_PLC_Kendra_Ayers.pdf")

In [2]:
text

'NON-DISCLOSURE AGREEMENT (NDA)\nThis Nondisclosure Agreement ("Agreement") is made on August 23, 2024 by and between\nthe following parties:\nDisclosing Party: Ali PLC\nAddress: 247 Clark Knoll, Deanberg, MO 12412\nReceiving Party: Kendra Ayers\nAddress: 7382 Christine Parkways, Georgeborough, VT 42010\n1. Definition of Confidential Information: Confidential Information includes all\ninformation or material that has, or could have, commercial value.\n2. Exclusions from Confidential Information: Information that is publicly known at the\ntime of disclosure is not considered confidential.\n3. Obligations of Receiving Party: The Receiving Party agrees to maintain confidentiality\nand not disclose the information.\n4. Time Periods: This Agreement remains in effect until the information is no longer\nconfidential or upon mutual consent.\nIN WITNESS WHEREOF, the parties have executed this Agreement as of the date first\nwritten above.\nDisclosing Party\'s Signature: ________________________

In [3]:
analyzer = AnalyzerEngine()
anonymizer = AnonymizerEngine()

results = analyzer.analyze(text=text, language="en")

anonymized_result = anonymizer.anonymize(text=text, analyzer_results=results)

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text) 

/opt/anaconda3/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



🔹 Anonymized Document:

NON-<IN_PAN> AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on <DATE_TIME> by and between
the following parties:
<IN_PAN> Party: Ali PLC
Address: 247 <PERSON>, <LOCATION>, MO 12412
Receiving Party: <PERSON>
Address: 7382 <PERSON>, <LOCATION>, VT 42010
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligations of Receiving Party: The Receiving Party agrees to maintain confidentiality
and not disclose the information.
4. Time Periods: This Agreement remains in effect until the information is no longer
confidential or upon mutual consent.
IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first
written above.
Disclosing Party's Signature: __________________________
Receiving Party's

In [4]:
nlp_config = {
    "nlp_engine_name": "spacy",
    "models": [{"lang_code": "en", "model_name": "en_core_web_lg"}]  # Proper format
}

nlp_engine = NlpEngineProvider(nlp_configuration=nlp_config).create_engine()

analyzer = AnalyzerEngine(nlp_engine=nlp_engine)
anonymizer = AnonymizerEngine()

results = analyzer.analyze(text=text, language="en")

anonymized_result = anonymizer.anonymize(text=text, analyzer_results=results)

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Anonymized Document:

NON-<IN_PAN> AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on <DATE_TIME> by and between
the following parties:
<IN_PAN> Party: Ali PLC
Address: 247 <PERSON>, <LOCATION>, MO 12412
Receiving Party: <PERSON>
Address: 7382 <PERSON>, <LOCATION>, VT 42010
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligations of Receiving Party: The Receiving Party agrees to maintain confidentiality
and not disclose the information.
4. Time Periods: This Agreement remains in effect until the information is no longer
confidential or upon mutual consent.
IN WITNESS WHEREOF, the parties have executed this Agreement as of the date first
written above.
Disclosing Party's Signature: __________________________
Receiving Party's

In [5]:
stanza_nlp = stanza.Pipeline(lang="en", processors="tokenize,ner")

doc = stanza_nlp(text)

stanza_results = [
    RecognizerResult(entity_type=ent.type, start=ent.start_char, end=ent.end_char, score=0.85)
    for ent in doc.ents
]

anonymizer = AnonymizerEngine()
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=stanza_results)

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)

2025-03-06 20:23:25 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2025-03-06 20:23:25 INFO: Downloaded file to /Users/alicja/stanza_resources/resources.json
2025-03-06 20:23:25 WARNING: Language en package default expects mwt, which has been added
2025-03-06 20:23:26 INFO: Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

2025-03-06 20:23:26 INFO: Using device: cpu
2025-03-06 20:23:26 INFO: Loading: tokenize
2025-03-06 20:23:26 INFO: Loading: mwt
2025-03-06 20:23:26 INFO: Loading: ner
2025-03-06 20:23:28 INFO: Done loading processors!



🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (<ORG>)
<LAW> (<LAW>) is made on <DATE> by and between
the following parties:
Disclosing Party: Ali PLC
Address: <CARDINAL> <PERSON>, <GPE>, MO 12412
Receiving Party: <PERSON>
Address: <CARDINAL> Christine Parkways, <GPE>, VT 42010
<CARDINAL>. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
<CARDINAL>. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
<CARDINAL>. Obligations of <ORG>: <ORG> agrees to maintain confidentiality
and not disclose the information.
<CARDINAL>. Time Periods: This Agreement remains in effect until the information is no longer
confidential or upon mutual consent.
IN WITNESS WHEREOF, the parties have executed this Agreement as of the date <ORDINAL>
written above.
Disclosing Party's Signature: __________________________
Receiving Party'

In [6]:
tagger = SequenceTagger.load("flair/ner-english-large")  

sentence = Sentence(text)
tagger.predict(sentence)

flair_results = [
    {"entity_type": entity.tag, "start": entity.start_position, "end": entity.end_position, "text": entity.text}
    for entity in sentence.get_spans("ner")
]

print("\n🔹 Flair Detected PII:\n")
for entity in flair_results:
    print(f"{entity['entity_type']}: {entity['text']} (Start: {entity['start']}, End: {entity['end']})")

presidio_results = [
    RecognizerResult(entity_type=entity["entity_type"], start=entity["start"], end=entity["end"], score=0.85)
    for entity in flair_results
]

anonymizer = AnonymizerEngine()

anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)

2025-03-06 20:23:43,579 SequenceTagger predicts: Dictionary with 20 tags: <unk>, O, S-ORG, S-MISC, B-PER, E-PER, S-LOC, B-ORG, E-ORG, I-PER, S-PER, B-MISC, I-MISC, E-MISC, I-ORG, B-LOC, E-LOC, I-LOC, <START>, <STOP>

🔹 Flair Detected PII:

MISC: Agreement (Start: 50, End: 59)
ORG: Ali PLC (Start: 157, End: 164)
LOC: Clark Knoll (Start: 178, End: 189)
LOC: Deanberg (Start: 191, End: 199)
LOC: MO (Start: 201, End: 203)
ORG: Kendra Ayers (Start: 227, End: 239)
LOC: Christine Parkways (Start: 254, End: 272)
LOC: Georgeborough (Start: 274, End: 287)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This Nondisclosure <MISC> ("Agreement") is made on August 23, 2024 by and between
the following parties:
Disclosing Party: <ORG>
Address: 247 <LOC>, <LOC>, <LOC> 12412
Receiving Party: <ORG>
Address: 7382 <LOC>, <LOC>, VT 42010
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions fro

In [7]:
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

results = ner_pipeline(text)

print("\n🔹 Detected PII:\n")
for entity in results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in results
]

anonymizer = AnonymizerEngine()

anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII:

ORG: NDA (Score: 0.71)
MISC: Non (Score: 0.92)
MISC: Agreement (Score: 0.73)
MISC: Agreement (Score: 0.61)
PER: Ali (Score: 0.69)
ORG: P (Score: 0.71)
PER: Clark K (Score: 0.91)
ORG: ##noll (Score: 0.50)
PER: Dean (Score: 0.98)
ORG: ##berg (Score: 0.31)
PER: Kendra Aye (Score: 0.99)
PER: Christine (Score: 0.78)
LOC: Parkway (Score: 0.36)
PER: George (Score: 0.39)
LOC: ##borough (Score: 0.95)
LOC: V (Score: 0.66)
ORG: Re (Score: 0.59)
ORG: Party (Score: 0.88)
ORG: Receiving Party (Score: 0.91)
ORG: Disc (Score: 0.71)
ORG: Party (Score: 0.88)
ORG: Receiving Party (Score: 0.76)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (<ORG>)
This <MISC>disclosure <MISC> ("<MISC>") is made on August 23, 2024 by and between
the following parties:
Disclosing Party: <PER> <ORG>LC
Address: 247 <PER><ORG>, <PER><ORG>, MO 12412
Receiving Party: <PER>rs
Address: 7382 <PER> <LOC>s, <PER><LOC>, <LOC>T 42010
1. Definition of Confidential Information: Confidential Information includes all


In [8]:
ner_pipeline = pipeline("ner", model="Jean-Baptiste/roberta-large-ner-english", aggregation_strategy="simple")
results = ner_pipeline(text)

print("\n🔹 Detected PII:\n")
for entity in results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in results
]

anonymizer = AnonymizerEngine()

anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII:

MISC: DA (Score: 0.51)
ORG:  Ali PLC (Score: 1.00)
LOC:  Clark Knoll (Score: 0.99)
LOC:  Deanberg (Score: 1.00)
LOC:  MO (Score: 1.00)
PER:  Kendra Ayers (Score: 0.99)
LOC:  Christine Parkways (Score: 0.93)
LOC:  Georgeborough (Score: 0.98)
LOC:  VT (Score: 1.00)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (N<MISC>)
This Nondisclosure Agreement ("Agreement") is made on August 23, 2024 by and between
the following parties:
Disclosing Party: <ORG>
Address: 247 <LOC>, <LOC>, <LOC> 12412
Receiving Party: <PER>
Address: 7382 <LOC>, <LOC>, <LOC> 42010
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligations of Receiving Party: The Receiving Party agrees to maintain confidentiality
and not disclose the information.
4. Time Period

In [9]:
ner_pipeline = pipeline("ner", model="Jean-Baptiste/roberta-large-ner-english", aggregation_strategy="simple")

results = ner_pipeline(text)

print("\n🔹 Detected PII (Score > 0.8):\n")
filtered_results = [entity for entity in results if entity["score"] > 0.8]

for entity in filtered_results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in filtered_results
]

anonymizer = AnonymizerEngine()

# Apply anonymization
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

# Print final anonymized text
print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII (Score > 0.8):

ORG:  Ali PLC (Score: 1.00)
LOC:  Clark Knoll (Score: 0.99)
LOC:  Deanberg (Score: 1.00)
LOC:  MO (Score: 1.00)
PER:  Kendra Ayers (Score: 0.99)
LOC:  Christine Parkways (Score: 0.93)
LOC:  Georgeborough (Score: 0.98)
LOC:  VT (Score: 1.00)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on August 23, 2024 by and between
the following parties:
Disclosing Party: <ORG>
Address: 247 <LOC>, <LOC>, <LOC> 12412
Receiving Party: <PER>
Address: 7382 <LOC>, <LOC>, <LOC> 42010
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligations of Receiving Party: The Receiving Party agrees to maintain confidentiality
and not disclose the information.
4. Time Periods: This Agree

In [10]:
ner_pipeline = pipeline("ner", model="dslim/bert-base-NER", aggregation_strategy="simple")

nlp = spacy.load("en_core_web_lg")

transformer_results = ner_pipeline(text)

filtered_transformer_results = [
    entity for entity in transformer_results if entity["score"] > 0.8
]

spacy_results = []
for ent in nlp(text).ents:
    if ent.label_ in ["DATE", "TIME", "EMAIL", "MONEY", "PHONE_NUMBER"]:
        spacy_results.append({
            "entity_group": ent.label_,
            "start": ent.start_char,
            "end": ent.end_char,
            "word": ent.text,
            "score": 1.0  
        })

all_results = filtered_transformer_results + spacy_results

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in all_results
]

anonymizer = AnonymizerEngine()
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Detected PII (Transformer + spaCy):\n")
for entity in all_results:
    print(f"{entity['entity_group']}: {entity['word']} (Start: {entity['start']}, End: {entity['end']})")

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)

/opt/anaconda3/lib/python3.12/site-packages/spacy/util.py:910: UserWarning: [W095] Model 'en_core_web_lg' (3.7.1) was trained with spaCy v3.7.2 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)



🔹 Detected PII (Transformer + spaCy):

MISC: Non (Start: 36, End: 39)
PER: Clark K (Start: 178, End: 185)
PER: Dean (Start: 191, End: 195)
PER: Kendra Aye (Start: 227, End: 237)
LOC: ##borough (Start: 280, End: 287)
ORG: Party (Start: 612, End: 617)
ORG: Receiving Party (Start: 623, End: 638)
ORG: Party (Start: 938, End: 943)
DATE: August 23, 2024 (Start: 85, End: 100)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This <MISC>disclosure Agreement ("Agreement") is made on <DATE> by and between
the following parties:
Disclosing Party: Ali PLC
Address: 247 <PER>noll, <PER>berg, MO 12412
Receiving Party: <PER>rs
Address: 7382 Christine Parkways, George<LOC>, VT 42010
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligations of Receiving <ORG

In [11]:
ner_pipeline = pipeline("ner", model="ab-ai/pii_model", aggregation_strategy="simple")

results = ner_pipeline(text)

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in results  
]

anonymizer = AnonymizerEngine()
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Detected PII (All Results):\n")
for entity in results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII (All Results):

DATE: august 23, 2024 (Score: 0.99)
BUILDINGNUMBER: 247 (Score: 0.99)
STREET: clark knoll, (Score: 1.00)
CITY: deanberg, mo (Score: 0.94)
ZIPCODE: 12412 (Score: 0.98)
FIRSTNAME: kendra (Score: 1.00)
LASTNAME: ayers (Score: 1.00)
BUILDINGNUMBER: 7382 (Score: 1.00)
STREET: christine parkways, (Score: 1.00)
CITY: georgeborough, (Score: 0.99)
STATE: vt (Score: 0.83)
ZIPCODE: 42010 (Score: 0.99)
ETHEREUMADDRESS: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ (Score: 0.49)
ETHEREUMADDRESS: _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ _ (Score: 0.45)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on <DATE> by and between
the following parties:
Disclosing Party: Ali PLC
Address: <BUILDINGNUMBER> <STREET> <CITY> <ZIPCODE>
Receiving Party: <FIRSTNAME> <LASTNAME>
Address: <BUILDINGNUMBER> <STREET> <CITY> <STATE> <ZIPCODE>
1. Definition of Confidential Information: Confidential Information includes all
inform

In [12]:
ner_pipeline = pipeline("ner", model="hewonty/bert-ner-finetuned-pii", aggregation_strategy="simple")

results = ner_pipeline(text)

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in results  
]

anonymizer = AnonymizerEngine()
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Detected PII (All Results from `hewonty/bert-ner-finetuned-pii`):\n")
for entity in results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII (All Results from `hewonty/bert-ner-finetuned-pii`):

NAME_STUDENT: Ali PLC (Score: 0.85)
STREET_ADDRESS: 247 Clark Knoll, Deanberg, MO 12412 (Score: 0.97)
NAME_STUDENT: Kendra (Score: 0.90)
NAME_STUDENT: ##rs (Score: 0.58)
STREET_ADDRESS: 7382 Christine Parkways, Georgeborough, VT 42010 (Score: 0.98)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on August 23, 2024 by and between
the following parties:
Disclosing Party: <NAME_STUDENT>
Address: <STREET_ADDRESS>
Receiving Party: <NAME_STUDENT> Aye<NAME_STUDENT>
Address: <STREET_ADDRESS>
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligations of Receiving Party: The Receiving Party agrees to maintain confidentiality
and no

In [13]:
ner_pipeline = pipeline("ner", model="iiiorg/piiranha-v1-detect-personal-information", aggregation_strategy="simple")

results = ner_pipeline(text)

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in results 
]

anonymizer = AnonymizerEngine()
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Detected PII (All Results from `iiiorg/piiranha-v1-detect-personal-information`):\n")
for entity in results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII (All Results from `iiiorg/piiranha-v1-detect-personal-information`):

BUILDINGNUM: 247 (Score: 1.00)
STREET: Clark Knoll (Score: 0.93)
CITY: Deanberg (Score: 0.85)
ZIPCODE: MO 12412 (Score: 0.86)
GIVENNAME: Kendra (Score: 0.50)
BUILDINGNUM: 7382 (Score: 1.00)
STREET: Christine Parkways (Score: 1.00)
CITY: Georgeborough (Score: 1.00)
ZIPCODE: VT 42010 (Score: 0.98)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on August 23, 2024 by and between
the following parties:
Disclosing Party: Ali PLC
Address:<BUILDINGNUM><STREET>,<CITY>,<ZIPCODE>
Receiving Party:<GIVENNAME> Ayers
Address:<BUILDINGNUM><STREET>,<CITY>,<ZIPCODE>
1. Definition of Confidential Information: Confidential Information includes all
information or material that has, or could have, commercial value.
2. Exclusions from Confidential Information: Information that is publicly known at the
time of disclosure is not considered confidential.
3. Obligation

In [14]:
ner_pipeline = pipeline("ner", model="h2oai/deberta_finetuned_pii", aggregation_strategy="simple")

results = ner_pipeline(text)

presidio_results = [
    RecognizerResult(entity_type=entity["entity_group"], start=entity["start"], end=entity["end"], score=entity["score"])
    for entity in results  
]

anonymizer = AnonymizerEngine()
anonymized_result = anonymizer.anonymize(text=text, analyzer_results=presidio_results)

print("\n🔹 Detected PII (All Results from `h2oai/deberta_finetuned_pii`):\n")
for entity in results:
    print(f"{entity['entity_group']}: {entity['word']} (Score: {entity['score']:.2f})")

print("\n🔹 Anonymized Document:\n")
print(anonymized_result.text)


🔹 Detected PII (All Results from `h2oai/deberta_finetuned_pii`):

DATE:  August 23, 2024 (Score: 1.00)
MIDDLENAME:  Ali (Score: 0.56)
SUFFIX:  P (Score: 0.90)
SUFFIX: LC (Score: 0.82)
STREETADDRESS:  247 Clark Knoll (Score: 1.00)
CITY:  Dean (Score: 1.00)
CITY: berg (Score: 1.00)
STATE:  MO (Score: 1.00)
ZIPCODE:  12412 (Score: 0.79)
FIRSTNAME:  Kend (Score: 0.98)
FIRSTNAME: ra (Score: 0.97)
LASTNAME:  Ay (Score: 0.74)
LASTNAME: ers (Score: 0.85)
STREETADDRESS:  7 (Score: 1.00)
STREETADDRESS: 382 Christine Parkways (Score: 1.00)
CITY:  George (Score: 1.00)
CITY: borough (Score: 1.00)
STATE:  VT (Score: 0.99)
ZIPCODE:  4 (Score: 0.99)
ZIPCODE: 2010
1 (Score: 1.00)
LASTNAME:  WH (Score: 0.55)
LASTNAME: ER (Score: 0.37)
MIDDLENAME: EO (Score: 0.34)
MIDDLENAME: F (Score: 0.13)

🔹 Anonymized Document:

NON-DISCLOSURE AGREEMENT (NDA)
This Nondisclosure Agreement ("Agreement") is made on<DATE> by and between
the following parties:
Disclosing Party:<MIDDLENAME><SUFFIX><SUFFIX>
Address:<STREET